<a href="https://colab.research.google.com/github/markusloecher/DataScience2021/blob/main/TWSM/Class4_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
TWSM_path = "/content/drive/MyDrive/teaching/TWSM/WorkInClass/"

#from TWSM import *
from tensorflow import keras
#from tensorflow.keras import layers
import pandas as pd
import numpy as np
from scipy.linalg import svd

### SVD

#### Tasks

1. Compute the SVD for our movie rankings and find the first two eigenvectors for movies and users. Note on preprocessing: 
  * Keep all users with less than 3 missing values and replace those with the mean ranking per user
  * Keep all movies with at least 8 rankings
2. Compute the SVD for the MNIST data
3. Reconstruct the digits with a "truncated SVD that captures 80% of the variance"


**Minimal Example for the SVD**

Running this example first prints the defined 3×2 matrix, then the 3×3 U matrix, 2 element Sigma vector, and 2×2 V^T matrix elements calculated from the decomposition.

In [ ]:
# Singular-value decomposition
import numpy as np
from scipy.linalg import svd
# define a matrix
A = np.array([[1, 2], [3, 4], [5, 6]])
print(A)
# SVD
U, s, VT = svd(A)
print(U)
print(s)
print(VT)

[[1 2]
 [3 4]
 [5 6]]
[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
[9.52551809 0.51430058]
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


**Reconstruct Matrix from SVD**

The U, s, and V elements returned from the svd() cannot be multiplied directly.

The s vector must be converted into a diagonal matrix using the diag() function. By default, this function will create a square matrix that is n x n, relative to our original matrix. This causes a problem as the size of the matrices do not fit the rules of matrix multiplication, where the number of columns in a matrix must match the number of rows in the subsequent matrix.

After creating the square Sigma diagonal matrix, the sizes of the matrices are relative to the original m x n matrix that we are decomposing, as follows:

$U (m x m) . Sigma (n x n) . V^T (n x n)$

Where, in fact, we require:

$U (m x m) . Sigma (m x n) . V^T (n x n)$

We can achieve this by creating a new Sigma matrix of all zero values that is m x n (e.g. more rows) and populate the first n x n part of the matrix with the square diagonal matrix calculated via diag().



In [ ]:
# Reconstruct SVD

# create m x n Sigma matrix
Sigma = np.zeros((A.shape[0], A.shape[1]))
# populate Sigma with n x n diagonal matrix
Sigma[:A.shape[1], :A.shape[1]] = np.diag(s)
# reconstruct matrix
B = U.dot(Sigma.dot(VT))
print(B)

[[1. 2.]
 [3. 4.]
 [5. 6.]]


The above complication with the Sigma diagonal only exists with the case where m and n are not equal. The diagonal matrix can be used directly when reconstructing a square matrix, as follows.

### Truncated SVD

Data with a large number of features, such as more features (columns) than observations (rows) may be reduced to a smaller subset of features that are most relevant to the prediction problem.

The result is a matrix with a lower rank that is said to approximate the original matrix.

To do this we can perform an SVD operation on the original data and select the top k largest singular values in Sigma. These columns can be selected from Sigma and the rows selected from $V^T$.

An approximate B of the original vector A can then be reconstructed.

$$B = U . Sigma_k . V^T_k$$

In natural language processing, this approach can be used on matrices of word occurrences or word frequencies in documents and is called **Latent Semantic Analysis** or **Latent Semantic Indexing**.



In [ ]:
url = f'https://docs.google.com/spreadsheets/d/1Yf1WcX5QKIIjnBcp4zlYa9p9PM5ff3joqAzHXFHPp8A/edit#gid=0'
url_1 = url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(url_1)

print(df.shape)
df

(18, 22)


,First Name,Last Name,Princess Diaries,Dumb Dumber,Color Purple,Brave Heart,Narnia,Sense/Sensibility,LiarLiar,Django Unchained,...,SE7EN,The Intern,John Wick,Crazy Rich Asians,Call me by your Name,Gone Girl,Prisoners,Nightcrawler,Gran Torino,Million Dollar Baby
0,Jafar,Abdurrahmaan,3.0,2.0,4.0,2.0,2.0,3.0,3.0,4.0,...,4.0,5.0,5.0,5.0,5.0,4.0,4.0,3.0,NaN,NaN
1,Moritz Kai Phillip,Deecke,2.0,3.0,4.0,2.0,3.0,3.0,NaN,5.0,...,5.0,2.0,2.0,1.0,4.0,3.0,4.0,5.0,4.0,5.0
2,Huong,Duong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Felix,Eger,2.0,2.0,3.0,3.0,3.0,2.0,2.0,5.0,...,5.0,3.0,4.0,2.0,2.0,4.0,3.0,4.0,3.0,3.0
4,Johana,Estrada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Theresa,Ewert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Rukniya,Gurung,4.0,3.0,4.0,3.0,3.0,3.0,2.0,5.0,...,3.0,4.0,2.0,4.0,4.0,4.0,4.0,3.0,NaN,NaN
7,Lilit,Harutyunyan,3.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,...,5.0,5.0,3.0,4.0,5.0,5.0,5.0,3.0,3.0,NaN
8,Philipp,Heitmann,3.0,1.0,4.0,1.0,4.0,2.0,1.0,5.0,...,5.0,5.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,4.0
9,Minh Anh,Hoang,3.0,2.0,NaN,NaN,4.0,3.0,NaN,NaN,...,2.0,4.0,3.0,5.0,4.0,5.0,3.0,3.0,4.0,3.0


In [ ]:
#Toss out movie with less than 8 rankings:
#MissingMovie = np.sum(df.isna(), axis = 1)
df2 = df.dropna(axis=0, thresh=18)
print(df2.shape)
df3 = df2.dropna(axis=1, thresh=3)
print(df3.shape)
#df = df[np.where(MissingMovie==0)]
#MissingUsers = np.sum(df.isna(), axis = 0)

(10, 22)
(10, 22)


## Data

#### MNIST


In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [ ]:
# flatten data
x_train_flat = np.reshape(x_train, [60000,784])[:5000]

In [ ]:
x_train_flat.shape

(5000, 784)

In [ ]:
# SVD
U, s, VT = svd(x_train_flat)

In [ ]:
print(U.shape)
print(VT.shape)

(5000, 5000)
(784, 784)


In [ ]:
#s is a vector right now, so you need to convert it using 
#Sigma = np.diag(s)

m = x_train_flat.shape[0]
n = x_train_flat.shape[1]
Sigma = np.zeros((m,n))
# populate Sigma with n x n diagonal matrix
Sigma[:n, :n] = np.diag(s)
#full reconstruction: 
Xreconstruct = np.dot(U,np.dot(Sigma,VT))
Xreconstruct.shape


(5000, 784)

In [ ]:
#loss function:
MSE = np.mean((Xreconstruct - x_train_flat)**2)
MSE#Question: any randomness in the svd component ??

9.350605778717117e-09

In [ ]:
#truncated
k = 20#number of latent factors we keep
Sigmak = Sigma[:, :k]
VTk = VT[:k, :]
X = np.dot(U,np.dot(Sigmak,VTk))
Xreconstruct.shape

In [ ]:
print(U.shape)
print(VT.shape)

(5000, 5000)
(784, 784)


In [ ]:
# create m x n Sigma matrix
m= x_train_flat.shape[0]
n = x_train_flat.shape[1]
Sigma = np.zeros((m,n))

# populate Sigma with n x n diagonal matrix
Sigma[:n, :n] = np.diag(s)
#Sigma = np.diag(s)
Sigma.shape

(5000, 784)

In [ ]:
# select
n_elements = 20
Sigma = Sigma[:, :n_elements]
VT = VT[:n_elements, :]
# reconstruct
B = Sigma.dot(VT)
print(B.shape)
# transform
#T = U.dot(Sigma)
#print(T.shape)
#T = x_train_flat.dot(VT.T)
#print(T.shape)

(5000, 784)


In [ ]:
import matplotlib.pyplot as plt
